In [1]:
import os
import subprocess
from openpilot_tools.lib.logreader import LogReader
from openpilot_tools.lib.framereader import FrameReader
from scipy.signal import resample
import numpy as np
import cv2 as cv
from skimage.transform import resize
import h5py
import time
import shutil
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
import pandas as pd

In [2]:
preprocessing_dir = "./custom_data/preprocessing"
downscaled_dir = "./custom_data/downscaled"
raw_videos = os.listdir(preprocessing_dir)
resize_size = (50,160,2)
resize_frame_size = (50,160,3)

In [3]:
def filter_order_clips(raw_videos, clip_ends = 2):
    video_sets = {}
    clipped_videos = []
    for video in raw_videos:
        date_time = "--".join(video.split("--")[0:2])
        if date_time not in video_sets:
            video_sets[date_time] = 0
        video_sets[date_time] += 1
        
    for date_time, num_frags in sorted(video_sets.items()):
        for frag in range(num_frags):
            if (frag > clip_ends) and (frag < (num_frags - clip_ends)):
                video_loc = date_time + "--" + str(frag)
                clipped_videos.append(video_loc)
    return clipped_videos
clipped_videos = filter_order_clips(raw_videos)

In [4]:
def extract_logs(video_dir, frame_shape):
    lr = list(LogReader(video_dir + "/rlog.bz2"))
    speed_data = [l.carState.vEgo for l in lr if l.which() == 'carState']
    speed_data = np.array(speed_data)
    resampled_speeds = resample(speed_data, frame_shape)
    return resampled_speeds

In [5]:
def convert_video(downscaled_dir, video_dir):
    downscaled_vid = downscaled_dir + '/' + video_dir.split("-")[-1] + str(time.time()) + 'preprocessed.mp4'
    subprocess.call('ffmpeg -r 24 -i ' +  video_dir + '/fcamera.hevc' + 
                    ' -c:v libx265 -r 20 -filter:v scale=640:480 -crf 10 -c:a -i ' +
                    downscaled_vid, shell = True)
    return downscaled_vid

In [6]:
def opticalFlowDense(image_current, image_next):
    """
    Args:
        image_current : RGB image
        image_next : RGB image
    return:
        optical flow magnitude and angle and stacked in a matrix
    """
    image_current = np.array(image_current)
    image_next = np.array(image_next)
    gray_current = cv.cvtColor(image_current, cv.COLOR_RGB2GRAY)
    gray_next = cv.cvtColor(image_next, cv.COLOR_RGB2GRAY)
    flow = cv.calcOpticalFlowFarneback(gray_current, gray_next, None, 0.5, 1, 15, 2, 5, 1.3, 0)
    return flow

In [7]:
def augment(image_current, image_next):
    brightness = np.random.uniform(0.5, 1.5)
    img1 = ImageEnhance.Brightness(image_current).enhance(brightness)
    img2 = ImageEnhance.Brightness(image_next).enhance(brightness)            

    color = np.random.uniform(0.5, 1.5)
    img1 = ImageEnhance.Brightness(img1).enhance(color)
    img2 = ImageEnhance.Brightness(img2).enhance(color)
    return img1, img2

In [8]:
import random
def op_flow_video(preprocessed_video, augment_frames = True):
    op_flows = []
    frames = []
    count = 0
    vidcap = cv.VideoCapture(preprocessed_video)
    success,frame1 = vidcap.read()
    frame1 = cv.cvtColor(frame1, cv.COLOR_BGR2RGB)
    frame1 = Image.fromarray(frame1).crop((0, 170, 640, 370)).resize((160,50))
    while success:
        if (count % 100 == 0) and count > 0:
            print(count)
        success,frame2 = vidcap.read()
        if success == True:
            frame2 = cv.cvtColor(frame2, cv.COLOR_BGR2RGB)
            frame2 = Image.fromarray(frame2).crop((0, 170, 640, 370)).resize((160,50))
            if augment_frames == True:
                if random.random() > .85:
                    frame1, frame2 = augment(frame1, frame2)
            flow = opticalFlowDense(frame1, frame2)
            op_flows.append(flow)
            frames.append(np.array(frame2))
            frame1 = frame2
            count+=1
        else:
            print("video reading completed")
            continue
    return np.array(frames), np.array(op_flows), count


In [9]:
hdf5_path = './train_data/train.hdf5'


In [10]:
# '''Only use to reset dataset'''
# with h5py.File(hdf5_path, "w") as f:
#     f.create_dataset("frame", shape = (1, resize_frame_size[0], resize_frame_size[1], resize_frame_size[2]),
#                     maxshape = (None, resize_frame_size[0], resize_frame_size[1], resize_frame_size[2]), 
#                     chunks = (1, resize_frame_size[0], resize_frame_size[1], resize_frame_size[2]))
#     f.create_dataset("op_flow", shape = (1, resize_size[0], resize_size[1], resize_size[2]),
#                     maxshape = (None, resize_size[0], resize_size[1], resize_size[2]), 
#                     chunks = (1, resize_size[0], resize_size[1], resize_size[2]))
#     f.create_dataset("speed", shape = (1,), maxshape = (None,))
# f.close()

In [11]:
def write_hdf5(hdf5_path, frames, op_flows, resampled_speeds):
    with h5py.File(hdf5_path) as f:
        print(len(frames), len(op_flows), len(resampled_speeds))
        print(f["frame"], f["op_flow"], f["speed"])
        f["frame"].resize(f["frame"].len() + len(frames), axis = 0)
        f["op_flow"].resize(f["op_flow"].len() + len(op_flows), axis = 0)
        f["speed"].resize(f["speed"].len() + len(resampled_speeds), axis = 0)
        f["frame"][-len(frames):] = frames
        f["op_flow"][-len(op_flows):] = op_flows
        f["speed"][-len(resampled_speeds):] = resampled_speeds

In [12]:
def archive_processed(video_dir):
    processed_dir = video_dir.split("/")
    processed_dir[-2] = "processed"
    processed_dir = "/".join(processed_dir)
    shutil.move(video_dir, processed_dir)

In [16]:
#preprocess custom data
for video in raw_videos:
    video_dir = preprocessing_dir + "/" + video
    if video in clipped_videos:
        files = os.listdir(video_dir)
        if "fcamera.hevc" in files and "rlog.bz2" in files:
            downscaled_vid = convert_video(downscaled_dir, video_dir)
            frames, op_flows, count = op_flow_video(downscaled_vid)
            resampled_speeds = extract_logs(video_dir, frames.shape[0])
            write_hdf5(hdf5_path, frames, op_flows, resampled_speeds)
    archive_processed(video_dir)
        

100
200
300
400
500
600
700
800
900
1000
video reading completed
1001 1001 1001
<HDF5 dataset "frame": shape (103787, 50, 160, 3), type "<f4"> <HDF5 dataset "op_flow": shape (103787, 50, 160, 2), type "<f4"> <HDF5 dataset "speed": shape (103787,), type "<f4">
100
200
300
400
500
600
700
800
900
1000
video reading completed
1001 1001 1001
<HDF5 dataset "frame": shape (104788, 50, 160, 3), type "<f4"> <HDF5 dataset "op_flow": shape (104788, 50, 160, 2), type "<f4"> <HDF5 dataset "speed": shape (104788,), type "<f4">
100
200
300
400
500
600
700
800
900
1000
video reading completed
1001 1001 1001
<HDF5 dataset "frame": shape (105789, 50, 160, 3), type "<f4"> <HDF5 dataset "op_flow": shape (105789, 50, 160, 2), type "<f4"> <HDF5 dataset "speed": shape (105789,), type "<f4">
100
200
300
400
500
600
700
800
900
1000
video reading completed
1001 1001 1001
<HDF5 dataset "frame": shape (106790, 50, 160, 3), type "<f4"> <HDF5 dataset "op_flow": shape (106790, 50, 160, 2), type "<f4"> <HDF5 datase

100
200
300
400
500
600
700
800
900
1000
video reading completed
1001 1001 1001
<HDF5 dataset "frame": shape (135819, 50, 160, 3), type "<f4"> <HDF5 dataset "op_flow": shape (135819, 50, 160, 2), type "<f4"> <HDF5 dataset "speed": shape (135819,), type "<f4">
100
200
300
400
500
600
700
800
900
1000
video reading completed
1001 1001 1001
<HDF5 dataset "frame": shape (136820, 50, 160, 3), type "<f4"> <HDF5 dataset "op_flow": shape (136820, 50, 160, 2), type "<f4"> <HDF5 dataset "speed": shape (136820,), type "<f4">
100
200
300
400
500
600
700
800
900
1000
video reading completed
1001 1001 1001
<HDF5 dataset "frame": shape (137821, 50, 160, 3), type "<f4"> <HDF5 dataset "op_flow": shape (137821, 50, 160, 2), type "<f4"> <HDF5 dataset "speed": shape (137821,), type "<f4">
100
200
300
400
500
600
700
800
900
1000
video reading completed
1001 1001 1001
<HDF5 dataset "frame": shape (138822, 50, 160, 3), type "<f4"> <HDF5 dataset "op_flow": shape (138822, 50, 160, 2), type "<f4"> <HDF5 datase

In [27]:
# with open("./train_data/train.txt", "r") as f:
#     speeds = f.read()
# speeds = [float(i) for i in speeds.split("\n")]

In [30]:
# #preprocess comma train data
# comma_train_vid = './train_data/train.mp4'
# hdf5_path_comma_train = './train_data/comma_train.hdf5'

# '''Only use to reset dataset'''
# with h5py.File(hdf5_path_comma_train, "w") as f:
#     f.create_dataset("frame", shape = (1, resize_frame_size[0], resize_frame_size[1], resize_frame_size[2]),
#                     maxshape = (None, resize_frame_size[0], resize_frame_size[1], resize_frame_size[2]), 
#                     chunks = (1, resize_frame_size[0], resize_frame_size[1], resize_frame_size[2]))
#     f.create_dataset("op_flow", shape = (1, resize_size[0], resize_size[1], resize_size[2]),
#                     maxshape = (None, resize_size[0], resize_size[1], resize_size[2]), 
#                     chunks = (1, resize_size[0], resize_size[1], resize_size[2]))
#     f.create_dataset("speed", shape = (1,), maxshape = (None,))
# f.close()

# frames, op_flows, count = op_flow_video(comma_train_vid, augment_frames = False)
# write_hdf5(hdf5_path_comma_train, frames, op_flows, speeds)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [31]:
# #preprocess comma data
# comma_test_vid = './test_data/test.mp4'
# hdf5_path_comma_test = './test_data/comma_test.hdf5'

# '''Only use to reset dataset'''
# with h5py.File(hdf5_path_comma_test, "w") as f:
#     f.create_dataset("frame", shape = (1, resize_frame_size[0], resize_frame_size[1], resize_frame_size[2]),
#                     maxshape = (None, resize_frame_size[0], resize_frame_size[1], resize_frame_size[2]), 
#                     chunks = (1, resize_frame_size[0], resize_frame_size[1], resize_frame_size[2]))
#     f.create_dataset("op_flow", shape = (1, resize_size[0], resize_size[1], resize_size[2]),
#                     maxshape = (None, resize_size[0], resize_size[1], resize_size[2]), 
#                     chunks = (1, resize_size[0], resize_size[1], resize_size[2]))
#     f.create_dataset("speed", shape = (1,), maxshape = (None,))
# f.close()

# frames, op_flows, count = op_flow_video(comma_test_vid, augment_frames = False)
# null_speeds = np.zeros(count)
# write_hdf5(hdf5_path_comma_test, frames, op_flows, null_speeds)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
video reading completed
10797 10797 10797
<HDF5 dataset "frame": shape (1, 50, 160, 3), type "<f4"> <HDF5 dataset "op_flow": shape (1, 50, 160, 2), type "<f4"> <HDF5 dataset "speed": shape (1,), type "<f4">


In [ ]:
# def extract_video(video_dir):
#     extracting = True
#     count = 0
#     skipped_frames = 0
#     while extracting == True:
#         if count % 5 == 0:
#             count +=1
#             skipped_frames += 1
#         try:
#             fr = FrameReader(video_dir + '/fcamera.hevc')
#             fr.get(count, pix_fmt='rgb24')[0]
#             count += 1
#         except:
#             extracting = False
#     return count - skipped_frames
# extract_video(video_dir)